In [31]:
#TODO: generate hsi dummy data => run inference.py => will produce output.
#  then replace their data with my data in same folder structure and run inference.py again. Need to have data_stats.json and mapping.json as well

#current: error File "C:/Users/jedwinne/udt/Lib/site-packages/pytorch_lightning/core/saving.py", line 169, in _load_state
#     keys = obj.load_state_dict(checkpoint["state_dict"], strict=strict)
#            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
#   File "C:/Users/jedwinne/udt/Lib/site-packages/torch/nn/modules/module.py", line 2152, in load_state_dict
#     raise RuntimeError('Error(s) in loading state_dict for {}:/n/t{}'.format(
# RuntimeError: Error(s) in loading state_dict for GanCondinitionalDomainAdaptationINNHSI:
#         size mismatch for model.module_list.0.w_perm: copying a param with shape torch.Size([100, 100]) from checkpoint, the shape in current model is torch.Size([100]).


# https://www.spiedigitallibrary.org/journals/journal-of-biomedical-optics/volume-27/issue-07/074702/3D-printed-tissue-simulating-phantoms-for-near-infrared-fluorescence-imaging/10.1117/1.JBO.27.7.074702.full?SSO=1

# => check wat er in C:/Users/jedwinne/OneDrive - UGent/PhD-UG-8HYNGY3/Data/publication_data_dreher_DT/publication_data/simulated_data/HSI_Data/intermediates/semantic_v2/train
import pandas as pd
import numpy as np

# https://github.com/IMSY-DKFZ/UDT-cINN

# Data loading

In [28]:
def normalization(spectra):
    norms = np.linalg.norm(spectra,ord=1, axis=1,keepdims=True)
    spectra = spectra / norms    
    return spectra

folder = 'C:/Users/jedwinne/OneDrive - UGent/Documents/GitHub/PersonalProjects/Oxygenation_AI/real_data/'

#phantom
phantom = np.load(folder + 'tissue_phantoms.npy')
spectra_phantom = phantom[:,0:16]
label_phantom = phantom[:,-1]
spectra_phantom = normalization(spectra_phantom)
phantom_tissue_label = np.zeros(len(spectra_phantom))

def sample_per_label(spectra, labels, tissue_label, n_samples):
    unique_labels = np.unique(labels)
    indices = []

    for label in unique_labels:
        label_indices = np.where(labels == label)[0]
        sample_indices = np.random.choice(label_indices, n_samples, replace=False)
        indices.extend(sample_indices)

    sampled_spectra = spectra[indices]
    sampled_labels = labels[indices]
    sampled_tissue_label = tissue_label[indices]
    
    return sampled_spectra, sampled_labels, sampled_tissue_label

spectra_phantom, label_phantom, phantom_tissue_label = sample_per_label(spectra_phantom, label_phantom, phantom_tissue_label, 500)

#liver
liver = np.load(folder + 'liver.npy')
spectra_liver = liver[:,0:16]
spectra_liver = normalization(spectra_liver)
liver_tissue_label = np.ones(len(spectra_liver)) * 1

#heart
heart = np.load(folder + 'heart_wall.npy')
spectra_heart = heart[:,0:16]
spectra_heart = normalization(spectra_heart)
heart_tissue_label = np.ones(len(spectra_heart)) * 2

#coronary artery
ca= np.load(folder + 'coronary_artery.npy')
spectra_ca = ca[:,0:16]
spectra_ca = normalization(spectra_ca)
ca_tissue_label = np.ones(len(spectra_ca)) * 3

#GI tract
gi = np.load(folder + 'gi_tract.npy')
spectra_gi = gi[:,0:16]
spectra_gi = normalization(spectra_gi)
gi_tissue_label = np.ones(len(spectra_gi)) * 4

# my simulations
df = np.load('C:/Users/jedwinne/OneDrive - UGent/Documents/GitHub/PersonalProjects/Oxygenation_AI/dataframes_adapted/simulations_10k.npy')
simulation_spectra = df[:,0:16]
simulation_labels = df[:,16:]
simulation_spectra = normalization(simulation_spectra)

# Ayala
ayala_spectra = np.load('C:/Users/jedwinne/OneDrive - UGent/Documents/GitHub/PersonalProjects/Oxygenation_AI/dataframes_adapted/simulations_ayala.npy')
ayala_spectra = normalization(ayala_spectra)
ayala_labels = np.load('C:/Users/jedwinne/OneDrive - UGent/Documents/GitHub/PersonalProjects/Oxygenation_AI/dataframes_adapted/ayala_labels.npy') * 100

In [35]:
#split in train, test and validation sets

# synthetic data
from sklearn.model_selection import train_test_split
save_folder = "C:/Users/jedwinne/OneDrive - UGent/PhD-UG-8HYNGY3/Data/publication_data_dreher_DT/publication_data/simulated_data/HSI_Data/intermediates/semantic_v2_jens/"
train_data, temp_data = train_test_split(simulation_spectra, test_size=0.3, random_state=42)
np.save(save_folder + 'train_synthetic_unique/' + 'train.npy', train_data)

validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)
np.save(save_folder + 'val_synthetic_unique/' + 'val.npy', validation_data)
np.save(save_folder + 'test_synthetic_unique/' + 'test.npy', test_data)

#real data
real_data = np.concatenate((spectra_phantom, spectra_liver, spectra_heart, spectra_ca, spectra_gi), axis=0)
real_data_labels = np.concatenate((phantom_tissue_label, liver_tissue_label, heart_tissue_label, ca_tissue_label, gi_tissue_label), axis=0)

train_data, temp_data, = train_test_split(real_data, test_size=0.3, random_state=42)
np.save(save_folder + 'train/' + 'train.npy', train_data)
validation_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42)
np.save(save_folder + 'val/' + 'val.npy', validation_data)
np.save(save_folder + 'test/' + 'test.npy', test_data)